In [39]:
import platform
print(platform.python_version())


3.8.5


In [18]:
try:
    import mlagents
    from mlagents_envs.environment import UnityEnvironment as UE
    from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
    print("ml-agents already installed")
except ImportError:
    !pip install mlagents==0.26.0
    print("Installed ml-agents")

ml-agents already installed


In [52]:
# -----------------
# This code is used to close an env that might not have been closed before
try:
    env.close()
except:
    pass
# -----------------

In [53]:
# env = UE(file_name='run31', seed=1, side_channels=[])
channel = EngineConfigurationChannel()
env = UE("run31", base_port=5000, side_channels=[channel])
channel.set_configuration_parameters(time_scale = 20.0)

In [54]:
env.reset()

In [55]:
behavior_name = list(env.behavior_specs)[0]
print(f"Name of the behavior : {behavior_name}")
spec = env.behavior_specs[behavior_name]

Name of the behavior : Hummingbird?team=0


In [56]:
spec

BehaviorSpec(observation_specs=[ObservationSpec(shape=(44,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='RayPerceptionSensor'), ObservationSpec(shape=(3,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='StackingSensor_size3_VectorSensor_size1')], action_spec=ActionSpec(continuous_size=0, discrete_branches=(3, 3, 3)))

In [57]:
print("Number of observations : ", len(spec.observation_specs))

Number of observations :  2


In [27]:
# Is there a visual observation ?
# Visual observation have 3 dimensions: Height, Width and number of channels
vis_obs = any(len(spec.shape) == 3 for spec in spec.observation_specs)
print("Is there a visual observation ?", vis_obs)

Is there a visual observation ? False


In [28]:
# Is the Action continuous or multi-discrete ?
if spec.action_spec.continuous_size > 0:
    print(f"There are {spec.action_spec.continuous_size} continuous actions")
if spec.action_spec.is_discrete():
    print(f"There are {spec.action_spec.discrete_size} discrete actions")


# How many actions are possible ?
#print(f"There are {spec.action_size} action(s)")

# For discrete actions only : How many different options does each action has ?
if spec.action_spec.discrete_size > 0:
    for action, branch_size in enumerate(spec.action_spec.discrete_branches):
        print(f"Action number {action} has {branch_size} different options")
    


There are 3 discrete actions
Action number 0 has 3 different options
Action number 1 has 3 different options
Action number 2 has 3 different options


In [29]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [30]:
env.set_actions(behavior_name, spec.action_spec.empty_action(len(decision_steps)))

In [31]:
env.step()

In [32]:
import matplotlib.pyplot as plt
%matplotlib inline

for index, obs_spec in enumerate(spec.observation_specs):
    if len(obs_spec.shape) == 3:
        print("Here is the first visual observation")
        plt.imshow(decision_steps.obs[index][0,:,:,:])
        plt.show()

for index, obs_spec in enumerate(spec.observation_specs):
    if len(obs_spec.shape) == 1:
        print("First vector observations : ", decision_steps.obs[index][0,:])

First vector observations :  [0.         1.         0.         0.23652978 0.         1.
 0.         0.2127816  0.         1.         0.         0.28532526
 0.         1.         0.         0.20440233 0.         1.
 0.         0.24007748 0.         1.         0.         0.20857991
 0.         1.         0.         0.21439953 0.         1.
 0.         0.22667955 0.         1.         0.         0.204668
 0.         1.         0.         0.26548997 0.         1.
 0.         0.20757797]
First vector observations :  [0. 0. 0.]


In [33]:
len(decision_steps.obs)

2

In [34]:
decision_steps.obs

[array([[0.        , 1.        , 0.        , 0.23652978, 0.        ,
         1.        , 0.        , 0.2127816 , 0.        , 1.        ,
         0.        , 0.28532526, 0.        , 1.        , 0.        ,
         0.20440233, 0.        , 1.        , 0.        , 0.24007748,
         0.        , 1.        , 0.        , 0.20857991, 0.        ,
         1.        , 0.        , 0.21439953, 0.        , 1.        ,
         0.        , 0.22667955, 0.        , 1.        , 0.        ,
         0.204668  , 0.        , 1.        , 0.        , 0.26548997,
         0.        , 1.        , 0.        , 0.20757797]], dtype=float32),
 array([[0., 0., 0.]], dtype=float32)]

In [35]:
for obs in decision_steps.obs:
    print(len(obs[0]))

44
3


In [36]:
action = spec.action_spec.random_action(len(decision_steps))
action.discrete

array([[0, 1, 2]], dtype=int32)

In [51]:
for episode in range(1):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking 
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0] 

        # Generate an action for all agents
#         print(decision_steps[0])
        action = spec.action_spec.random_action(len(decision_steps))
        print(action.discrete)
        # Set the actions
        env.set_actions(behavior_name, action)

        # Move the simulation forward
        env.step()

        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

UnityEnvironmentException: No Unity environment is loaded.

In [38]:
env.close()
print("Closed environment")

Closed environment
